In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [2]:
# Define directories
base_dir = 'newdata'
train_dir = os.path.join(base_dir, 'training_set', 'training_set')
test_dir = os.path.join(base_dir, 'test_set', 'test_set')

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [3]:
# Normalize the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 8005 images belonging to 2 classes.


In [4]:
# Flow validation images in batches of 32 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 2023 images belonging to 2 classes.


In [5]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

C:\Users\Prashant\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=int(train_generator.samples/train_generator.batch_size),
    epochs=50,
    validation_data=test_generator,
    validation_steps=int(test_generator.samples/test_generator.batch_size)
)

Epoch 1/50


C:\Users\Prashant\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 131s 506ms/step - accuracy: 0.5207 - loss: 0.6919 - val_accuracy: 0.5491 - val_loss: 0.6886
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.5625 - loss: 0.3446 - val_accuracy: 0.4286 - val_loss: 0.3457
Epoch 3/50


C:\Users\Prashant\AppData\Local\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 300ms/step - accuracy: 0.5494 - loss: 0.6891 - val_accuracy: 0.6126 - val_loss: 0.6838
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.6250 - loss: 0.3440 - val_accuracy: 0.8571 - val_loss: 0.3218
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 141s 559ms/step - accuracy: 0.5686 - loss: 0.6831 - val_accuracy: 0.6151 - val_loss: 0.6764
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step - accuracy: 0.5938 - loss: 0.3399 - val_accuracy: 0.8571 - val_loss: 0.3171
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 144s 564ms/step - accuracy: 0.5757 - loss: 0.6808 - val_accuracy: 0.5427 - val_loss: 0.6772
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step - accuracy: 0.5000 - loss: 0.3374 - val_accuracy: 0.4286 - val_loss: 0.3444
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 143s 560ms/step - accuracy: 0.5859 - loss: 0.6741 - val_accuracy: 0.5193 - val_loss: 0.6901
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step - accuracy: 0.5625 - loss: 0.3362 - val

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

64/64 - 11s - 178ms/step - accuracy: 0.7133 - loss: 0.5728

Test accuracy: 0.7132970690727234


In [9]:
# Prediction function
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    return "Dog" if prediction[0][0] > 0.5 else "Cat"

# Example usage
image_path = r"PetImages\validation\26.jpg"
prediction = predict_image(model, image_path)
print("Prediction:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
Prediction: Cat
